In [66]:
!workon cv
!pip install requests

'workon' is not recognized as an internal or external command,
operable program or batch file.


In [67]:
###
# search_words = ["MR16"]
img_dir = "./Images/"

In [68]:
import urllib.parse

In [69]:
!pip install selenium
from selenium import webdriver

In [70]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [71]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
DRIVER_PATH = 'C:/Users/HP/.wdm/drivers/chromedriver/win32/84.0.4147.30/chromedriver.exe'
wd = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147


[WDM] - Driver [C:\Users\HP\.wdm\drivers\chromedriver\win32\84.0.4147.30\chromedriver.exe] found in cache


In [72]:
wd.get('https://google.com')


In [73]:
search_box = wd.find_element_by_css_selector('input.gLFyf')
search_box.send_keys('MR16')

In [74]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=3):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(60)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [75]:
import requests

In [76]:
import io
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [77]:
!pip install pillow

In [78]:
import os
import io
from PIL import Image
from io import StringIO

In [79]:
def search_and_download(search_term:str,driver_path:str,target_path='./Images',number_images=15):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=1)
        
    for elem in res:
        persist_image(target_folder,elem)

In [80]:
import time
import hashlib

In [81]:
search_term='MR16'
search_and_download(
    search_term=search_term,
    driver_path=DRIVER_PATH
)

Found: 100 search results. Extracting links from 0:100
Found: 17 image links, done!
SUCCESS - saved https://www.mygreenlighting.co.uk/images/products/white/35w_mr16_halogen_bulb_energy_saving_12v-1876-1000-2452.jpg - as ./Images\mr16\1defc997d6.jpg
SUCCESS - saved https://d114hh0cykhyb0.cloudfront.net/images/uploads/mr16-led-bulb-30-watt-equivalent-bi-pin-led-flood-light-bulb.jpg - as ./Images\mr16\e1523e92c1.jpg
SUCCESS - saved https://www.assets.signify.com/is/image/PhilipsLighting/75ed815b2f9946a983f1a8e800388efa?clipPathE=legacy_path&$pnglarge$ - as ./Images\mr16\ba6b3fe5cf.jpg
SUCCESS - saved https://cdn3.volusion.com/ohtjy.rszhx/v/vspfiles/photos/LED-6WMR16COB-2.jpg?v-cache=1525874998 - as ./Images\mr16\226d6579d1.jpg
SUCCESS - saved https://www.e-conolight.com/media/catalog/product/cache/163ca96bbb4a55bdf943cbd362c38761/l/t/ltg-mr16-60w-35d_lamp-front-45angle-view_6.jpg - as ./Images\mr16\6e0f595cd4.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcS66-